In [26]:
import json
import numpy as np
import pandas as pd

In [39]:
def modify(data, preds):
    new_data = []
    for k, instance in enumerate(preds):
        x, y = instance["beats"][0], instance["beats"][1]
        new_labels, new_times = [], []
        for l, t in zip(x, y):
            if t < 10:
                new_labels.append(l)
                new_times.append(t)
            else:
                break

        new_labels = np.array(new_labels).astype("float32") + 1
        new_times = np.array(new_times)
        sorted_indices = np.argsort(new_times)
        new_labels = new_labels[sorted_indices].tolist()
        new_times = new_times[sorted_indices].tolist()
        line = data[k]
        line["beats_predicted"] = [new_times, new_labels]
        new_data.append(line)
        
    return new_data

In [40]:
val_data = [json.loads(line) for line in open("data/100eval_combined_chatgpt_aug4.json").readlines()]
val_preds = [json.loads(line) for line in open("saved/beats-model/1687193940/val_preds_epoch_8.json").readlines()]

test_data = [json.loads(line) for line in open("data/100test_musiccaps_ep4.json").readlines()]
test_preds = [json.loads(line) for line in open("saved/beats-model/1687193940/test_preds_epoch_8.json").readlines()]

In [41]:
val_new_preds = modify(val_data, val_preds)
test_new_preds = modify(test_data, test_preds)

In [42]:
with open("results/beats-model/1687193940_eval_preds.json", "w") as f:
    for line in val_new_preds:
        f.write(json.dumps(line) + "\n")
        
with open("results/beats-model/1687193940_test_preds.json", "w") as f:
    for line in test_new_preds:
        f.write(json.dumps(line) + "\n")